In [1]:
from datetime import date
import awswrangler as wr
import pandas as pd

In [2]:
import boto3
boto3.setup_default_session(profile_name="quannguyen")

In [3]:
s3_path = "s3://upskills-landing-zone/secmaster/USSECMASTER_ALL20220525.XLS"

In [6]:
df = wr.s3.read_excel(s3_path, skiprows=4)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6144 entries, 1 to 6144
Data columns (total 51 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Symbol              6144 non-null   object        
 1   Name                6144 non-null   object        
 2   COMPANYNAME         6144 non-null   object        
 3   SymbolCountry       6144 non-null   object        
 4   CUSIP               6144 non-null   object        
 5   SEDOL               6144 non-null   object        
 6   ISIN                6144 non-null   object        
 7   COUNTRYID           6144 non-null   float64       
 8   EXCHANGECOUNTRY     6144 non-null   object        
 9   COUNTRY             6144 non-null   object        
 10  EXCHANGENAME        6144 non-null   object        
 11  FSSECTOR            6144 non-null   object        
 12  FSIND               6144 non-null   object        
 13  FSSECTORN           6144 non-null   float64       
 1

- Drop rows with full `nan`

In [28]:
df.dropna(how='all', inplace=True)

- Rename columns: 
    - 'SYMBOL' to 'SymbolCountry'
    - 'Unnamed: 49':  'Other'

In [10]:
df.rename(columns={"SYMBOL": "SymbolCountry",
                   "Unnamed: 49":  "Other"},
          inplace=True)

- There are 2 `DATETRADING` columns, so drop the one with more `nan` values
- convert column to `np.datetime` format

In [20]:
temp_df = df['DATETRADING.1'] if df['DATETRADING'].isna().sum() > df['DATETRADING.1'].isna().sum() else df['DATETRADING']

In [29]:
df.drop(columns=['DATETRADING', 'DATETRADING.1'], inplace=True)

In [32]:
df['DATETRADING'] = pd.to_datetime(temp_df, format="%Y%m%d")

- Convert `FISCALYEAREC` to `datetime` as well

In [35]:
df['FISCALYEAREC'] = pd.to_datetime(df['FISCALYEAREC'], format='%Y%m')

- Create partition columns

In [40]:
df['YEAR'] = df['DATETRADING'].dt.year
df['MONTH'] = df['DATETRADING'].dt.month